In [ ]:
import apogee.tools.read as apread
from apogee.tools import bitmask
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import window as wn
import os
import access_spectrum as acs
import reduce_dataset as rd
import polyfit as pf
import matplotlib
import apogee.spec.plot as splot
from read_clusterdata import read_caldata
# Show plots in notebook
%pylab inline
# Read in red clump sample
data = readcaldata()
# Set random seed value
seedval = 1
# Start random seed
np.random.seed(seedval) #original was 17
# Set global plot font preferences
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

In [ ]:
data['APOGEE_ID'] = data['ID']

In [ ]:
direc = './open_clusters/'
specsfile = direc+'specs_cdata.pkl'
errsfile = direc+'errs_cdata.pkl'
if os.path.isfile(specsfile):
    specs,ind = acs.pklread(specsfile)
    errs,ind = acs.pklread(errsfile)
    print 'Missing {0} stars'.format(len(ind))
else:
    specdata = acs.get_spectra_asp(dat,header = False)
    errdata= acs.get_spectra_asp(dat,ext = 2, header = False)
    acs.pklwrite(specsfile,specdata)
    acs.pklwrite(errsfile,errdata)
    specs,ind = specdata
    errs,ind = errdata  